In [1]:
%autosave 0

Autosave disabled


### Text generation using Project Gutenberg

In this tutorial, we will download a story from Project Gutenberg and use it to feed a text generation model.

About Project Gutenberg:
"Project Gutenberg offers over 56,000 free eBooks: Choose among free epub books, free kindle books, download them or read them online. You will find the world's great literature here, especially older works for which copyright has expired. We digitized and diligently proofread them with the help of thousands of volunteers."

Navigate to https://www.gutenberg.org/ and pick a book with a plain text file of around 200 kb (Alice in Wonderland is a good option: https://www.gutenberg.org/ebooks/11). Download the text as the basis for your model.

### Develop a Small LSTM Recurrent Neural Network
In this section we will develop a simple LSTM network to learn sequences of characters from Alice in Wonderland. In the next section we will use this model to generate new sequences of characters.

Let’s start off by importing the classes and functions we intend to use to train our model.

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Users\Hamburg\Anaconda3\envs\tensorflow2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Next, we need to load the ASCII text for the book into memory and convert all of the characters to lowercase to reduce the vocabulary that the network must learn.

In [10]:
import re
import sys

In [3]:
# load ascii text and covert to lowercase
filename = "data/dr_hyde_easy.txt"
raw_text = open(filename, encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text = re.sub(r'[^\w\s]',' ', raw_text)

# Get rid of line breaks and change multiple space to single
raw_text = re.sub(r'[\n]',' ', raw_text)
raw_text = re.sub(r' +', ' ', raw_text)

Now that the book is loaded, we must prepare the data for modeling by the neural network. We cannot model the characters directly, instead we must convert the characters to integers.

We can do this easily by first creating a set of all of the distinct characters in the book, then creating a map of each character to a unique integer.

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

You can see that there may be some characters that we could remove to further clean up the dataset that will reduce the vocabulary and may improve the modeling process.

Now that the book has been loaded and the mapping prepared, we can summarize the dataset.

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  151293
Total Vocab:  37


We now need to define the training data for the network. There is a lot of flexibility in how you choose to break up the text and expose it to the network during training.

In this tutorial we will split the book text up into subsequences with a fixed length of 100 characters, an arbitrary length. We could just as easily split the data up by sentences and pad the shorter sequences and truncate the longer ones.

Each training pattern of the network is comprised of 100 time steps of one character (X) followed by one character output (y). When creating these sequences, we slide this window along the whole book one character at a time, allowing each character a chance to be learned from the 100 characters that preceded it (except the first 100 characters of course).

In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  151193


Now that we have prepared our training data we need to transform it so that it is suitable for use with Keras.

First we must transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network.

Next we need to rescale the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network that uses the sigmoid activation function by default.

Finally, we need to convert the output patterns (single characters converted to integers) into a one hot encoding. This is so that we can configure the network to predict the probability of each of the different characters in the vocabulary (an easier representation) rather than trying to force it to predict precisely the next character. Each y value is converted into a sparse vector, full of zeros except with a 1 in the column for the letter (integer) that the pattern represents.


In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

We can now define our LSTM model. Here we define a single hidden LSTM layer with 256 memory units. The network uses dropout with a probability of 20. The output layer is a Dense layer using the softmax activation function to output a probability prediction for each of the characters between 0 and 1.

The problem is really a single character classification problem with n character classes and as such is defined as optimizing the log loss (cross entropy), here using the ADAM optimization algorithm for speed.


In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

There is no test dataset. We are modeling the entire training dataset to learn the probability of each character in a sequence.

We are not interested in the most accurate (classification accuracy) model of the training dataset. This would be a model that predicts each character in the training dataset perfectly. Instead we are interested in a generalization of the dataset that minimizes the chosen loss function. We are seeking a balance between generalization and overfitting but short of memorization.


In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
151168/151193 [============================>.] - ETA: 0s - loss: 2.7911
Epoch 00001: loss improved from inf to 2.79100, saving model to weights-improvement-01-2.7910.hdf5
151193/151193 [==============================] - 215s 1ms/step - loss: 2.7910
Epoch 2/20
151168/151193 [============================>.] - ETA: 0s - loss: 2.6379
Epoch 00002: loss improved from 2.79100 to 2.63793, saving model to weights-improvement-02-2.6379.hdf5
151193/151193 [==============================] - 214s 1ms/step - loss: 2.6379
Epoch 3/20
151168/151193 [============================>.] - ETA: 0s - loss: 2.5624
Epoch 00003: loss improved from 2.63793 to 2.56246, saving model to weights-improvement-03-2.5625.hdf5
151193/151193 [==============================] - 213s 1ms/step - loss: 2.5625
Epoch 4/20
151168/151193 [============================>.] - ETA: 0s - loss: 2.5021
Epoch 00004: loss improved from 2.56246 to 2.50216, saving model to weights-improvement-04-2.5022.hdf5
151193/151193 [===========

After running the example, you should have a number of weight checkpoint files in the local directory.

You can delete them all except the one with the smallest loss value, eg `weights-improvement-19-1.9435.hdf5`

### Generating Text with an LSTM Network
Generating text using the trained LSTM network is relatively straightforward.

Firstly, we load the data and define the network in exactly the same way, except the network weights are loaded from a checkpoint file and the network does not need to be trained.


In [13]:
# load the network weights
filename = "weights-improvement-20-1.9344.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Also, when preparing the mapping of unique characters to integers, we must also create a reverse mapping that we can use to convert the integers back to characters so that we can understand the predictions.


In [14]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

The simplest way to use the Keras LSTM model to make predictions is to first start off with a seed sequence as input, generate the next character then update the seed sequence to add the generated character on the end and trim off the first character. This process is repeated for as long as we want to predict new characters (e.g. a sequence of 1,000 characters in length).

We can pick a random input pattern as our seed sequence, then print generated characters as we generate them.


In [10]:
import sys

In [15]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print( "\nDone.")

Seed:
" ion and must flee before daylight from a house that was no longer mine and hurrying back to my cabin "
et io the mawe of the sare thi fane oo the fand of the saae whth a streng of toeer and the lawter sooe a creat dare oo the sereo of the fouse oh the saalen the lawter whsh a stidn and the lawyer saed the lawyer i sae io whet ie a mort an wou cane a auaat foro the coor of the sale the sawe oo the coor of the saalen the lawyer saed the lawyer i sae io whet ie a mort an wou cane a auaat foro the coor of the sale the sawe oo the coor of the saalen the lawyer saed the lawyer i sae io whet ie a mort an wou cane a auaat foro the coor of the sale the sawe oo the coor of the saalen the lawyer saed the lawyer i sae io whet ie a mort an wou cane a auaat foro the coor of the sale the sawe oo the coor of the saalen the lawyer saed the lawyer i sae io whet ie a mort an wou cane a auaat foro the coor of the sale the sawe oo the coor of the saalen the lawyer saed the lawyer i sae io whet ie a

What about the results shows good language generation? What is done poorly?

The model was able to create something like words in the beginning, however, afterwards it just picked up the same pattern over and over again.  One could assume that it did so because that pattern is common throughout the book or it isn't able to distinguish other patterns after those letters.

### Exercise 1: Try a bigger model

We will keep the number of memory units the same at 256, but add a second layer.

In [16]:
model2 = Sequential()
model2.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), 
                return_sequences=True))
model2.add(LSTM(256))
model2.add(Dense(y.shape[1], activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-mod2-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model2.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
151168/151193 [============================>.] - ETA: 0s - loss: 2.6894
Epoch 00001: loss improved from inf to 2.68936, saving model to weights-improvement-mod2-01-2.6894.hdf5
151193/151193 [==============================] - 485s 3ms/step - loss: 2.6894
Epoch 2/20
151168/151193 [============================>.] - ETA: 0s - loss: 2.4266
Epoch 00002: loss improved from 2.68936 to 2.42657, saving model to weights-improvement-mod2-02-2.4266.hdf5
151193/151193 [==============================] - 494s 3ms/step - loss: 2.4266
Epoch 3/20
151168/151193 [============================>.] - ETA: 0s - loss: 2.2216
Epoch 00003: loss improved from 2.42657 to 2.22155, saving model to weights-improvement-mod2-03-2.2215.hdf5
151193/151193 [==============================] - 487s 3ms/step - loss: 2.2215
Epoch 4/20
151168/151193 [============================>.] - ETA: 0s - loss: 2.0699
Epoch 00004: loss improved from 2.22155 to 2.06995, saving model to weights-improvement-mod2-04-2.0699.hdf5
151193

In [18]:
# load the network weights
filename = "weights-improvement-mod2-20-1.2442.hdf5"
model2.load_weights(filename)
model2.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model2.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print( "\nDone.")

Seed:
" th which i listened to the civilities of my unhappy victim i declare at least before god no man mora "
ly plt reterted the lawyer i shought it was not tn me a changed renlsee to the project gutenberg tm trademark and distribution of the project gutenberg tm micense and distribution or any pther with the perains of the fyl tere of the fyn tere aut the ward of the lawy moment i was still and i cegind the servant metter i shank you sile that i cannot sead the lawyer i have have his friends were the strange cloter was not my hiad to my consenoent for my sesvrn i dould have teen and i have have eone to my heart that i cannot sead the lawyer i have have his friends were the strange cloter was not my hiad to my consenoent for my sesvrn i dould have teen and i have have eone to my heart that i cannot sead the lawyer i have have his friends were the strange cloter was not my hiad to my consenoent for my sesvrn i dould have teen and i have have eone to my heart that i cannot sead the law

Again, what about the results shows good language generation? What is done poorly?

This time the model is able to better predict some words, such as **lawyer** and **friends**.  However, it seems to replace some letters such as **s** for **r** and **s** for **t** in words such as *sead* that should be *read* and *shank* instead of *thank*.  However, there still seems to be repitition in the model in what it predicts.

### Exercise 2: Create a word based model

Instead of generating language character by character, let's build an word generating model.

In [4]:
word_text = raw_text.split(' ')

# create mapping of unique chars to integers
words = sorted(list(set(word_text)))
word_to_int = dict((c, i) for i, c in enumerate(words))

n_words = len(word_text)
n_word_vocab = len(words)
print("Total Characters: ", n_words)
print("Total Vocab: ", n_word_vocab)

# prepare the dataset of input to output pairs encoded as integers
word_seq_length = 20
dataX_word = []
dataY_word = []
for i in range(0, n_words - word_seq_length, 1):
    seq_in = word_text[i:i + word_seq_length]
    seq_out = word_text[i + word_seq_length]
    dataX_word.append([word_to_int[word] for word in seq_in])
    dataY_word.append(word_to_int[seq_out])
n_word_patterns = len(dataX_word)
print("Total Patterns: ", n_word_patterns)

# reshape X to be [samples, time steps, features]
X_word = numpy.reshape(dataX_word, (n_word_patterns, word_seq_length, 1))
# normalize
X_word = X_word / float(n_word_vocab)
# one hot encode the output variable
y_word = np_utils.to_categorical(dataY_word)

Total Characters:  25927
Total Vocab:  3931
Total Patterns:  25907


In [6]:
model3 = Sequential()
model3.add(GRU(200, input_shape=(X_word.shape[1], X_word.shape[2]), 
                return_sequences=True))
model3.add(Dropout(.2))
model3.add(GRU(200))
model3.add(Dropout(.2))
model3.add(Dense(y_word.shape[1], activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-wordmod-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model3.fit(X_word, y_word, epochs=350, batch_size=128, callbacks=callbacks_list)

Epoch 1/350
25856/25907 [============================>.] - ETA: 0s - loss: 6.7273
Epoch 00001: loss improved from inf to 6.72783, saving model to weights-improvement-wordmod-01-6.7278.hdf5
25907/25907 [==============================] - 20s 758us/step - loss: 6.7278
Epoch 2/350
25856/25907 [============================>.] - ETA: 0s - loss: 6.4670
Epoch 00002: loss improved from 6.72783 to 6.46721, saving model to weights-improvement-wordmod-02-6.4672.hdf5
25907/25907 [==============================] - 18s 709us/step - loss: 6.4672
Epoch 3/350
25856/25907 [============================>.] - ETA: 0s - loss: 6.4352
Epoch 00003: loss improved from 6.46721 to 6.43536, saving model to weights-improvement-wordmod-03-6.4354.hdf5
25907/25907 [==============================] - 19s 730us/step - loss: 6.4354
Epoch 4/350
25856/25907 [============================>.] - ETA: 0s - loss: 6.4309
Epoch 00004: loss improved from 6.43536 to 6.43092, saving model to weights-improvement-wordmod-04-6.4309.hdf5
2

Epoch 33/350
25856/25907 [============================>.] - ETA: 0s - loss: 5.9531
Epoch 00033: loss improved from 5.98456 to 5.95358, saving model to weights-improvement-wordmod-33-5.9536.hdf5
25907/25907 [==============================] - 18s 708us/step - loss: 5.9536
Epoch 34/350
25856/25907 [============================>.] - ETA: 0s - loss: 5.9174
Epoch 00034: loss improved from 5.95358 to 5.91670, saving model to weights-improvement-wordmod-34-5.9167.hdf5
25907/25907 [==============================] - 19s 735us/step - loss: 5.9167
Epoch 35/350
25856/25907 [============================>.] - ETA: 0s - loss: 5.8784
Epoch 00035: loss improved from 5.91670 to 5.87818, saving model to weights-improvement-wordmod-35-5.8782.hdf5
25907/25907 [==============================] - 19s 725us/step - loss: 5.8782
Epoch 36/350
25856/25907 [============================>.] - ETA: 0s - loss: 5.8431
Epoch 00036: loss improved from 5.87818 to 5.84298, saving model to weights-improvement-wordmod-36-5.843

25856/25907 [============================>.] - ETA: 0s - loss: 3.4330
Epoch 00093: loss improved from 3.45679 to 3.43308, saving model to weights-improvement-wordmod-93-3.4331.hdf5
25907/25907 [==============================] - 18s 698us/step - loss: 3.4331
Epoch 94/350
25856/25907 [============================>.] - ETA: 0s - loss: 3.4099
Epoch 00094: loss improved from 3.43308 to 3.41009, saving model to weights-improvement-wordmod-94-3.4101.hdf5
25907/25907 [==============================] - 18s 689us/step - loss: 3.4101
Epoch 95/350
25856/25907 [============================>.] - ETA: 0s - loss: 3.3993
Epoch 00095: loss improved from 3.41009 to 3.39874, saving model to weights-improvement-wordmod-95-3.3987.hdf5
25907/25907 [==============================] - 18s 697us/step - loss: 3.3987
Epoch 96/350
25856/25907 [============================>.] - ETA: 0s - loss: 3.3657
Epoch 00096: loss improved from 3.39874 to 3.36534, saving model to weights-improvement-wordmod-96-3.3653.hdf5
25907/

Epoch 124/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.8908
Epoch 00124: loss improved from 2.90543 to 2.89138, saving model to weights-improvement-wordmod-124-2.8914.hdf5
25907/25907 [==============================] - 18s 694us/step - loss: 2.8914
Epoch 125/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.8752
Epoch 00125: loss improved from 2.89138 to 2.87512, saving model to weights-improvement-wordmod-125-2.8751.hdf5
25907/25907 [==============================] - 18s 690us/step - loss: 2.8751
Epoch 126/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.8664
Epoch 00126: loss improved from 2.87512 to 2.86661, saving model to weights-improvement-wordmod-126-2.8666.hdf5
25907/25907 [==============================] - 18s 683us/step - loss: 2.8666
Epoch 127/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.8530
Epoch 00127: loss improved from 2.86661 to 2.85341, saving model to weights-improvement-wordmod-1

Epoch 156/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.5452
Epoch 00156: loss improved from 2.56466 to 2.54499, saving model to weights-improvement-wordmod-156-2.5450.hdf5
25907/25907 [==============================] - 18s 685us/step - loss: 2.5450
Epoch 157/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.5370
Epoch 00157: loss improved from 2.54499 to 2.53715, saving model to weights-improvement-wordmod-157-2.5371.hdf5
25907/25907 [==============================] - 18s 688us/step - loss: 2.5371
Epoch 158/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.5269
Epoch 00158: loss improved from 2.53715 to 2.52702, saving model to weights-improvement-wordmod-158-2.5270.hdf5
25907/25907 [==============================] - 18s 690us/step - loss: 2.5270
Epoch 159/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.5403
Epoch 00159: loss did not improve
25907/25907 [==============================] - 18s 687us/step -

Epoch 224/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.1504
Epoch 00224: loss improved from 2.15408 to 2.15072, saving model to weights-improvement-wordmod-224-2.1507.hdf5
25907/25907 [==============================] - 18s 690us/step - loss: 2.1507
Epoch 225/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.1454
Epoch 00225: loss improved from 2.15072 to 2.14601, saving model to weights-improvement-wordmod-225-2.1460.hdf5
25907/25907 [==============================] - 18s 684us/step - loss: 2.1460
Epoch 226/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.1388
Epoch 00226: loss improved from 2.14601 to 2.13874, saving model to weights-improvement-wordmod-226-2.1387.hdf5
25907/25907 [==============================] - 18s 688us/step - loss: 2.1387
Epoch 227/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.1199
Epoch 00227: loss improved from 2.13874 to 2.12024, saving model to weights-improvement-wordmod-2

Epoch 260/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.0213
Epoch 00260: loss improved from 2.02868 to 2.02144, saving model to weights-improvement-wordmod-260-2.0214.hdf5
25907/25907 [==============================] - 18s 685us/step - loss: 2.0214
Epoch 261/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.0318
Epoch 00261: loss did not improve
25907/25907 [==============================] - 18s 682us/step - loss: 2.0317
Epoch 262/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.0294
Epoch 00262: loss did not improve
25907/25907 [==============================] - 18s 685us/step - loss: 2.0293
Epoch 263/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.0291
Epoch 00263: loss did not improve
25907/25907 [==============================] - 18s 687us/step - loss: 2.0291
Epoch 264/350
25856/25907 [============================>.] - ETA: 0s - loss: 2.0482
Epoch 00264: loss did not improve
25907/25907 [===========

25856/25907 [============================>.] - ETA: 0s - loss: 1.9467
Epoch 00298: loss did not improve
25907/25907 [==============================] - 18s 684us/step - loss: 1.9468
Epoch 299/350
25856/25907 [============================>.] - ETA: 0s - loss: 1.9314
Epoch 00299: loss did not improve
25907/25907 [==============================] - 18s 689us/step - loss: 1.9315
Epoch 300/350
25856/25907 [============================>.] - ETA: 0s - loss: 1.9207
Epoch 00300: loss improved from 1.91976 to 1.91945, saving model to weights-improvement-wordmod-300-1.9195.hdf5
25907/25907 [==============================] - 18s 691us/step - loss: 1.9195
Epoch 301/350
25856/25907 [============================>.] - ETA: 0s - loss: 1.9043
Epoch 00301: loss improved from 1.91945 to 1.90358, saving model to weights-improvement-wordmod-301-1.9036.hdf5
25907/25907 [==============================] - 18s 693us/step - loss: 1.9036
Epoch 302/350
25856/25907 [============================>.] - ETA: 0s - loss: 1

Epoch 336/350
25856/25907 [============================>.] - ETA: 0s - loss: 1.8624
Epoch 00336: loss did not improve
25907/25907 [==============================] - 18s 686us/step - loss: 1.8626
Epoch 337/350
25856/25907 [============================>.] - ETA: 0s - loss: 1.9104
Epoch 00337: loss did not improve
25907/25907 [==============================] - 18s 694us/step - loss: 1.9107
Epoch 338/350
25856/25907 [============================>.] - ETA: 0s - loss: 1.8494
Epoch 00338: loss did not improve
25907/25907 [==============================] - 18s 681us/step - loss: 1.8493
Epoch 339/350
25856/25907 [============================>.] - ETA: 0s - loss: 1.8509
Epoch 00339: loss did not improve
25907/25907 [==============================] - 18s 692us/step - loss: 1.8515
Epoch 340/350
25856/25907 [============================>.] - ETA: 0s - loss: 1.8358
Epoch 00340: loss improved from 1.84281 to 1.83632, saving model to weights-improvement-wordmod-340-1.8363.hdf5
25907/25907 [===========

In [7]:
# load the network weights
filename = "weights-improvement-wordmod-329-1.8428.hdf5"
model3.load_weights(filename)
model3.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
int_to_word = dict((i, c) for i, c in enumerate(words))

In [11]:
# pick a random seed
start = numpy.random.randint(0, len(dataX_word)-1)
pattern = dataX_word[start]
print( "Seed:")
print( "\"", ' '.join([int_to_word[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_word_vocab)
    prediction = model3.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print( "\nDone.")

Seed:
" the fellow had a key and what s more he has it still i saw him use it not a "
week ago the he with sir very i i was i with stay i the the mr a the the you of in and and he to with have of whom that relate of the the i of a he could so look and the taking of was the above very the an guarantee and tempted in and were it to it and and to looking of the of and part and of he and the know god only been sunday trade of of i undignified get a the will it a to the of the like almost he on 
Done.


### Exercise 3: Use a different source corpus for training and compare results

Pick a very different source corpus, like the King James Bible or something that would differ greatly from the book you initially chose.